In [1]:
import os
import numpy as np
import faiss
import nest_asyncio
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from llama_index.core.objects import ObjectIndex
#from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from typing import List, Optional

In [ ]:
llama-index
llama-index-llms-huggingface
llama-index-embeddings-fastembed
fastembed
Unstructured[md]
chromadb
llama-index-vector-stores-chroma
llama-index-llms-groq
einops
accelerate
sentence-transformers
llama-index-llms-mistralai
llama-index-llms-openai

In [2]:
nest_asyncio.apply() #birden fazla olay döngüsünü aynı anda çalıştırabilmek için 

In [16]:
import ollama
from llama_index.core import Settings
ollama.pull('llama3.1')
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.1")

embed_model = OllamaEmbeddings(model="llama3.1")
Settings.embed_model=embed_model
Settings.chunk_size=4096
print(llm)

Ollama
Params: {'model': 'llama3.1', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': None, 'repeat_last_n': None, 'repeat_penalty': None, 'temperature': None, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': None}, 'system': None, 'template': None, 'keep_alive': None, 'raw': None}


In [41]:
import os
from llama_index.llms.mistralai import MistralAI

os.environ["MISTRAL_API_KEY"] = "7lmdNQ5tjQOojlANOjUwxWlwjCrVgwZx"
llm = MistralAI(model="mistral-large-latest")
print(llm)


callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x0000022C3AD9A450> system_prompt=None messages_to_prompt=<function messages_to_prompt at 0x0000022BE1945C60> completion_to_prompt=<function default_completion_to_prompt at 0x0000022BE19FC9A0> output_parser=None pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'> query_wrapper_prompt=None model='mistral-large-latest' temperature=0.1 max_tokens=512 timeout=120.0 max_retries=5 random_seed=None additional_kwargs={}


In [17]:
from chromadb.config import Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
db = chromadb.PersistentClient(path="./chroma_db_mistral") #kalıcı veritabanı istemcisi
#belirtilen isimde bir koleksiyonu veritabanında arar mevcutsa erişim sağlar, yoksa yeni üretir
chroma_collection = db.get_or_create_collection("multidocument-agent")# koleksiyona veri ekleme ve çekme için kullanılır 
vector_store = ChromaVectorStore(chroma_collection=chroma_collection) #vektör depolama ve sssorgulama
storage_context = StorageContext.from_defaults(vector_store=vector_store)#varsayılan StorageContext oluşturulur

# Storage context oluşturma


In [18]:
#dokümanları alan dosya fonksiyonu tanımlıyoruz  temelde vektör sorgulama ve özetlem işlemini yapacak
def get_doc_tools(file_path:str,name:str)->str:
  '''
  get vector query and sumnmary query tools from a document
  '''
  #load documents
  documents = SimpleDirectoryReader(input_files = [file_path]).load_data()
  print(f"length of nodes")
  splitter = SentenceSplitter(chunk_size=4096,chunk_overlap=100) #bölme ve örtüşme ayarlama(bilgipaylaşımı)
  nodes = splitter.get_nodes_from_documents(documents)
  print(f"Length of nodes : {len(nodes)}")
  #düğümler vektör deposuna dönüştürülür
  vector_index = VectorStoreIndex(nodes,storage_context=storage_context)
  vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")
  #kalıcı  olarak saklanır
  # verilen sorguyu vektör deposunda arama 
  def vector_query(query:str,page_numbers:Optional[List[str]]=None)->str:
    '''
    perform vector search over index on
    query(str): query string needs to be embedded
    page_numbers(List[str]): list of page numbers to be retrieved,
                            leave blank if we want to perform a vector search over all pages
    '''
    page_numbers = page_numbers or []
    metadata_dict = [{"key":'page_label',"value":p} for p in page_numbers]
    #sorgu mototru
    query_engine = vector_index.as_query_engine(similarity_top_k =5,
                                                llm=llm,
                                                filters = MetadataFilters.from_dicts(metadata_dict,
                                                                                     condition=FilterCondition.OR)
                                                )
    #
    response = query_engine.query(query)
    return response
  #
  #sorgu aracı
  vector_query_tool = FunctionTool.from_defaults(name=f"vector_tool_{name}",
                                                fn=vector_query)
  # özet motoru
  summary_index = SummaryIndex(nodes)
  summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",
                                                       se_async=True,
                                                       llm=llm)  #fonksiyon çağırma API'si
  #özet aracı
  summary_query_tool = QueryEngineTool.from_defaults(name=f"summary_tool_{name}",
                                                     query_engine=summary_query_engine,
                                                    description=("Use ONLY IF you want to get a holistic summary of the documents."
                                                "DO NOT USE if you have specified questions over the documents."))
  return vector_query_tool,summary_query_tool

In [19]:
import os
root_path = r"C:\Users\bc199\Staj\data"
file_name = []
file_path = []
# kök dizindeki pdf  formatındaki dosyaların isim ve yollarını saklamak için
for files in os.listdir(root_path):  
  if files.endswith(".pdf"):
    file_name.append(files.split(".")[0])
    file_path.append(os.path.join(root_path,files))
#
print(file_name)
print(file_path)


['29398286213056125074', 'Kurumsal_Basvuru_Sozlesmesi_NESUE_2 _0_vs1_0 ', 'ltd_anasozlesme', 'ORNEK_MAL-SATIS-SOZLESMESI', 'Sozlesme-Taslagi-1']
['C:\\Users\\bc199\\Staj\\data\\29398286213056125074.pdf', 'C:\\Users\\bc199\\Staj\\data\\Kurumsal_Basvuru_Sozlesmesi_NESUE_2 _0_vs1_0 .pdf', 'C:\\Users\\bc199\\Staj\\data\\ltd_anasozlesme.pdf', 'C:\\Users\\bc199\\Staj\\data\\ORNEK_MAL-SATIS-SOZLESMESI.pdf', 'C:\\Users\\bc199\\Staj\\data\\Sozlesme-Taslagi-1.pdf']


In [20]:
#toplanan dosya isim ve yolları ile her bir dosya için vector ve summary aracı oluşturur
#bu araçlar sözllükte depolanır
papers_to_tools_dict = {} #anahtar dosya adı değer = araç (tool)
for name,filename in zip(file_name,file_path): # eşleştirme 
  vector_query_tool,summary_query_tool = get_doc_tools(filename,name)
  papers_to_tools_dict[name] = [vector_query_tool,summary_query_tool]

length of nodes
Length of nodes : 4
length of nodes
Length of nodes : 5
length of nodes
Length of nodes : 5
length of nodes
Length of nodes : 4
length of nodes
Length of nodes : 7


In [21]:
#oluşan tüm vektör ve özet araçları tek bir listede  toplanır
initial_tools = [t for f in file_name for t in papers_to_tools_dict[f]]
initial_tools

In [22]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex
#belirli nesneleri initialtools olrak kullanarak bir nesne indexi oluşturur 
obj_index = ObjectIndex.from_objects(initial_tools,index_cls=VectorStoreIndex)
#verilen nesneleri yani araçları indeksler ve nesneler üzerinde arama ve sorgulama yapmamızı sağlar 
#bir geri alma işlemcisine dönüştürürüz 
obj_retriever = obj_index.as_retriever(similarity_top_k=2)
#sorgu için aram yapar ve en benzer araçları içeren bir liste döndürür
tools = obj_retriever.retrieve("compare and contrast the papers self rag and corrective rag")
#
print(tools[0].metadata)
print(tools[1].metadata)





In [25]:
#bir geri alma işlemcisine dönüştürürüz 
obj_retriever = obj_index.as_retriever(similarity_top_k=2)
#sorgu için aram yapar ve en benzer araçları içeren bir liste döndürür
tools = obj_retriever.retrieve("compare and contrast the papers self rag and corrective rag")
#
print(tools[0].metadata)
print(tools[1].metadata)




ToolMetadata(description='Use ONLY IF you want to get a holistic summary of the documents.DO NOT USE if you have specified questions over the documents.', name='summary_tool_Kurumsal_Basvuru_Sozlesmesi_NESUE_2 _0_vs1_0 ', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)
ToolMetadata(description='Use ONLY IF you want to get a holistic summary of the documents.DO NOT USE if you have specified questions over the documents.', name='summary_tool_ORNEK_MAL-SATIS-SOZLESMESI', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)


In [15]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Summarize the paper BERT.")
print(response)

c:\Users\bc199\anaconda3\envs\lcenv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Based on the provided text snippet, here's a summary of the paper BERT:

The paper "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" introduces the BERT (Bidirectional Encoder Representations from Transformers) model, which achieves state-of-the-art results on various natural language processing tasks. The authors demonstrate that pre-training a large-scale multilingual masked language modeling task can be fine-tuned for downstream NLP tasks, such as question answering, sentiment analysis, and text classification.

The key contributions of the paper are:

1. **Pre-training**: BERT is pre-trained on a large-scale multilingual corpus to learn general-purpose language representations.
2. **Fine-tuning**: The pre-trained model can be fine-tuned for specific downstream tasks with only a few epochs of training, using a batch size of 32 and a learning rate of 5e-5 or lower.
3. **Large model size benefits**: BERT LARGE (340M parameters) outperforms BERT BASE (1

In [20]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[1].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Summarize the paper crag.")
print(response)

Based on the provided context, here's a summary of the paper "CRAG":

The paper presents a novel method called CRAG (Corrective Retrieval-Augmented Generation) that aims to improve the robustness of generation in retrieval-augmented generation (RAG) models. CRAG incorporates corrective strategies to handle incorrect or ambiguous retrieval results, using a lightweight retrieval evaluator to estimate relevance scores and trigger corresponding actions.

The proposed approach consists of three main components: a retriever, an evaluator, and a generator. The retriever uses a pre-trained model to retrieve relevant information from internal knowledge (i.e., the question-answering dataset itself), external knowledge (i.e., Google Search API), and combined knowledge. The evaluator scores the relevance of the retrieved information using a fine-tuned model.

CRAG demonstrates improvements in accuracy on four benchmark datasets (PopQA, Biography, PubHealth, and Arc-Challenge) compared to existing 

In [17]:
def custom_agent(query: str):
    # Retrieve the tools using the query
    tools = obj_retriever.retrieve(query)
    
    if tools:
        # Perform the vector search with the first tool
        vector_response = tools[0].query_engine.query(query)
        
        # Ensure the vector_response is in string format before summarizing
        if not isinstance(vector_response, str):
            vector_response = str(vector_response)  # Convert to string if necessary
        
        # Now use the summary tool to summarize the result of the vector search
        summary_response = tools[1].query_engine.query(vector_response)
        
        # Combine both responses or decide how to present them
        combined_response = f"Vector Search Result:\n{vector_response}\n\nSummary:\n{summary_response}"
    else:
        combined_response = "No relevant tools found."
    
    return combined_response

# Run the query and get the response
response = custom_agent("Summarize the paper BERT.")
print(response)


Vector Search Result:
Based on the provided text, here is a summary of the paper BERT:

The paper presents a new pre-training approach called Masked Language Modeling (MLM) for natural language processing tasks. The authors compare their model to another popular pre-trained model, OpenAI GPT.

The BERT model uses a multi-layer bidirectional Transformer architecture and is trained on large amounts of text data with two specific pre-training objectives: MLM and Next Sentence Prediction (NSP). Unlike other models, BERT does not use language modeling as its primary training task, but rather as one part of the overall pre-training process.

The authors evaluate their model on a suite of tasks including sentiment analysis, question answering, text classification, and others. They demonstrate that their pre-trained model outperforms many other state-of-the-art models on these tasks, even when trained with limited data.

Some key findings from the paper include:

* Removing NSP hurts performan

In [19]:
print(tools[0])
print(tools[1])

In [52]:
from llama_index.core.agent import FunctionCallingAgentWorker # ajan sınıfı verilen araçlar ve dil modeli ile sorgu 
from llama_index.core.agent import AgentRunner # ajanı çalıştırmak ve sorguları işlemek için
#işlev çağırma yeteneğine sahip bir ajan oluşturuyoruz 
agent_worker = FunctionCallingAgentWorker.from_tools(tool_retriever=obj_retriever,
                                                     llm=llm,
                                                     system_prompt="""You are an agent designed to answer queries over a set of given papers.
                                                     Please always use the tools provided to answer a question.Do not rely on prior knowledge.""",
                                                     verbose=True)
agent = AgentRunner(agent_worker) #agentworker'ı çalıştırabiliriz 

In [55]:

response = agent.query("Summarize the paper corrective BERT.")
print(str(response))

Added user message to memory: Summarize the paper corrective BERT.
=== Calling Function ===
Calling function: summary_tool_BERT_arxiv with args: {"input": "corrective BERT"}
=== Function Output ===
Based on the provided text snippet and the inferred context, a possible answer could be:

Ablation study for masking procedures.

This is an educated guess, and it may not be entirely accurate without further context. However, given that there's no specific mention of "corrective BERT" in the text, this answer attempts to relate to the general concept of "correction" or "adjustment" in the context of pre-training or fine-tuning a BERT model.
=== LLM Response ===
The summary of the paper corrective BERT is:

Ablation study for masking procedures.

This is an educated guess, and it may not be entirely accurate without further context. However, given that there's no specific mention of "corrective BERT" in the text, this answer attempts to relate to the general concept of "correction" or "adjus

In [22]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Kaç tane belge var?")
print(response)

According to the provided information, there are actually multiple answers to this question, which may seem confusing.

From the first source:

"There is no specific information about the number of documents in the provided snippet."

This suggests that the answer is unknown or not specified.

However, from other sources:

* "When looking at the existing documents, it can be seen that there are 17 articles or research papers." (Source 2)
* "There is only 1 document (file) mentioned..." (Source 3)

Taking all this into account, I would say that the most accurate answer to the query "Kaç tane belge var?" (How many documents are there?) is:

**Multiple answers: 1, 17, or unknown**

However, if you want a single final answer without considering the uncertainty of the first source:

The final answer is: $\boxed{17}$


In [23]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("BERT belgesinin içeriği nedir ?")
print(response)

Cevap: BERT belgesinin içeriği ile ilgili olarak, büyük ölçekli bir dil modeli geliştirme, doğal dil işleme ve komut istemcisinin yanıtlama gibi alanlarla ilgilidir.

Açıklayan metin, BERT modeli ile ilgili ayrıntıları ve pre-training, fine-tuning süreçlerini anlatıyor.

Peki BERT ne demektir?

Bert, İngilizce kelimeleri anlamak için kullanılan derin öğrenme teknolojisidir.


In [50]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Sözlesme icerigi nedir?")
print(response)

Sözleşme içeriği, DEMİR GIDA İNŞAAT NAKLİYE SANAYİ VE TİCARET LİMİTED ŞİRKETİ'nin kuruluşu, şirketin unvanı, amaç ve konusu, şirketin merkezi ve şubeleri, süresi, sermayesi, ilanlar, şirketin idaresi, temsil, hesap dönemi, karın dağıtımı, ihtiyat akçesi ve kanuni hükümleri gibi konuları içermektedir. Sözleşme, şirketin kurucularının adları, soyadları, ikametgahları ve uyruklarını da belirtmektedir. Ayrıca, şirketin faaliyet alanları, yurt içi ve yurt dışı faaliyetleri, finansal yükümlülükleri ve diğer hukuki ve idari konuları da kapsamaktadır.


In [52]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Kac maddde var?")
print(response)

13 madde var.


In [55]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Yasal hukumlulukler nedir?")
print(response)

Bu anasözleşmede bulunmayan hususlar hakkında Türk Ticaret Kanunu hükümleri uygulanır.


In [56]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Sermayesi ne kadar?")
print(response)

Şirketin sermayesi 5.000,00 TL'dir.


In [11]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response

# Sorguyu çalıştırma ve yanıtı alma
response = custom_agent("Hukuksal yukumlulukler nedir?")
print(response)

c:\Users\bc199\anaconda3\envs\lcenv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Hukuksal yükümlülükler şirketin merkezi ve adresi, süresi, sermayesi ve sermaye taahhütleri ile ilgilidir.


In [15]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
    else:
        response = "No relevant tools found."
    
    return response


response = custom_agent("Where is the company headquarters?")
print(response)

Based on the provided context, it's not possible to determine the exact location of the company headquarters. The article mentioned ("şirket merkezinin bulunduğu yerde") implies that the headquarters is located in Turkey, but the specific city or region cannot be inferred from the given information.

However, based on the query "Eskişehir'dir.", which seems to be a statement about the company being in Eskişehir, it can be inferred that:

Answer: Eskişehir.


In [29]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
        print(tools[0])
    else:
        response = "No relevant tools found."
    
    return response


response = custom_agent("Kac adet belge var?")
print(response)


İki adet belge var.


In [30]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
        print(tools[0])
    else:
        response = "No relevant tools found."
    
    return response


response = custom_agent("Ornek mal satis sozlesmesini özetler misin?")
print(response)


Ornek mal satiş sözlesi; taraflarin hak ve yükümlüklerini düzenleyen bir sözleşme. Alıcı satılan malzemelere gereken özen ve ihtimamı göstermek zorundadır. Taşıma sırasında malların sigorta edilmesi gereken koşullarda alıcı sorumludur. Tedarikçinin iflası halinde sözleşe feshedilir. Tüm uyuşmazlıklar TOBBUYUM Arabuluculuk ve Uyuşmazlık Çözüm Merkezi'nde arabuluculuk yoluyla çözümlenecektir.


In [31]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
        print(tools[0])
    else:
        response = "No relevant tools found."
    
    return response


response = custom_agent("Sozlesme taslagi 1'deki taraflar kimlerdir?")
print(response)


İdare (Dr. Mehmet YAVUZ olarak temsil edilir) ve Yüklenici'dir.


In [34]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
        print(tools[0])
    else:
        response = "No relevant tools found."
    
    return response


response = custom_agent("ltd_anasozlesme kac maddeden olusur?")
print(response)


Cevap 13


In [36]:
def custom_agent(query: str):
    # Araçları geri alma
    tools = obj_retriever.retrieve(query)
    
    # İlk aracı kullanarak sorguyu gerçekleştirme
    if tools:
        response = tools[0].query_engine.query(query)
        print(tools[0])
    else:
        response = "No relevant tools found."
    
    return response


response = custom_agent("sozlesme_taslagi_1 kac maddeden olusur?")
print(response)


Bu soru biraz karmaşıktır!

Her iki kaynak da Sozlesme_Taslagi_1.pdf dosyasını farklı sayıda maddeyle tanır:

* İlk kaynak, dosyanın 15 maddeden oluştuğunu söyler.
* İkinci kaynak, dosyanın 20 maddeden oluştuğunu söyler.

Ama üçüncü kaynak, dosyanın aslında 25 (yirmibeş) maddenin olduğunu belirtir.

Bu durumda, en doğru cevabı bulmak için elimizde bir tercih götürmez. Her üç kaynağın farklı olduğu görülmektedir.

Ancak, her kaynakın tek tek doğru olabileceği düşünülürse, cevapları birleştirmek için en uygun yol, sonuç olarak 25 maddeden oluşur. Bu, Sozlesme_Taslagi_1.pdf dosyası 15 ve 20 maddeden oluşma iddialarının aslında doğru olmamasını veya yanlış bilgiler olduğunu kanıtlamaktadır.

Sonuç olarak:
Sozlesme_Taslagi_1.pdf dosyası 25 (yirmibeş) maddeden oluşur.
